In [1]:
%matplotlib inline 
import matplotlib.pylab
import pandas as pd
import numpy as np

In [8]:
with open('data/ao_monthly.txt') as f:
    for x in range(5):
        print(next(f))

<_io.TextIOWrapper name='data/ao_monthly.txt' mode='r' encoding='cp1252'>
 1950    1  -0.60310E-01

 1950    2   0.62681E+00

 1950    3  -0.81275E-02

 1950    4   0.55510E+00

 1950    5   0.71577E-01



In [9]:
data = pd.read_fwf('data/ao_monthly.txt', header = None)

In [11]:
# Not so great
data[0:13]

,0,1,2
0,1950,1,-0.060310
1,1950,2,0.626810
2,1950,3,-0.008128
3,1950,4,0.555100
4,1950,5,0.071577
5,1950,6,0.538570
6,1950,7,-0.802480
7,1950,8,-0.851010
8,1950,9,0.357970
9,1950,10,-0.378900


### Look at the options for read_fwf...what looks relevant?

In [145]:
data = pd.read_fwf('data/ao_monthly.txt', parse_dates={'Date':[0,1]}, 
                   index_col = 0, header=None)

In [158]:
# %load snippets/readtime.py
data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, 
                   parse_dates = [[0, 1]], infer_datetime_format = True)

In [159]:
data.head()

,2
0_1,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


In [203]:
type(data.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [180]:
data.index

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01',
               ...
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01'],
              dtype='datetime64[ns]', name='Month', length=798, freq=None)

In [163]:
# data.index = data.index.strftime(date_format='%b %Y')
# doing this reformats data.index's type to string index -- messes up order e.g. min, max

In [164]:
data.index.names = ['Month']
data.columns = ['O2 Value']
data.head()

,O2 Value
Month,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


### What is the empirical range of dates?

In [166]:
# %load snippets/daterange.py
print(min(data.index))
print(max(data.index))

1950-01-01 00:00:00
2016-06-01 00:00:00


### How can we convert to complementary representation?

In [224]:
# %load snippets/changerep.py
data_p = data.to_period().head()
data_p

,O2 Value
Month,
1950-01,-0.060310
1950-02,0.626810
1950-03,-0.008128
1950-04,0.555100
1950-05,0.071577


In [228]:
print(type(data.index))
data.index

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01',
               ...
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01'],
              dtype='datetime64[ns]', name='Month', length=798, freq=None)

In [229]:
# This one has 'freq'
print(type(data_p.index))
data_p.index

<class 'pandas.core.indexes.period.PeriodIndex'>


PeriodIndex(['1950-01', '1950-02', '1950-03', '1950-04', '1950-05'], dtype='period[M]', name='Month', freq='M')

### More about reading in data

When reading in dates with a pd.read function, you have several time-related parameters you can adjust: 
parse_dates, infer_datetime_format, date_parser

Experiment with these using %timeit to see if there are performance differences

Hint:
infer_datetime_format = True, no date parser provided
What other combos can you come up with?

In [230]:
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')
dateparse(2010,2)

datetime.datetime(2010, 2, 1, 0, 0)

In [251]:
import timeit
# First, let's see how to use a date_parser:
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')

print('Not using a parseer, not using infer_datetime_format')
%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = None, infer_datetime_format=False)

print('\nNot using a parseer, using infer_datetime_format')
%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = None, infer_datetime_format=True)

print('\nUsing a parseer, not using infer_datetime_format')
%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = dateparse, infer_datetime_format=False)

print('\nUsing a parseer, using infer_datetime_format')
%timeit data = pd.read_fwf('data/ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = dateparse, infer_datetime_format=True)

Not using a parseer, not using infer_datetime_format
21.1 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Not using a parseer, using infer_datetime_format
22.5 ms ± 725 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Using a parseer, not using infer_datetime_format
66.4 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Using a parseer, using infer_datetime_format
68.9 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [249]:
data.head()

,O2 Value
Month,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


### You can also extract datetimes from existing Data Frame columns

In [232]:
# new in pandas 0.18
df = pd.DataFrame({'year':[2015, 2016], 'month':[2,3], 'day':[4,5], 'hour':[12, 13]})
df

,day,hour,month,year
0,4,12,2,2015
1,5,13,3,2016


In [233]:
pd.to_datetime(df)

0   2015-02-04 12:00:00
1   2016-03-05 13:00:00
dtype: datetime64[ns]

### Truncating

In [234]:
ts = pd.Series(range(10), index = pd.date_range('7/31/15', freq = 'M', periods = 10))
ts

2015-07-31    0
2015-08-31    1
2015-09-30    2
2015-10-31    3
2015-11-30    4
2015-12-31    5
2016-01-31    6
2016-02-29    7
2016-03-31    8
2016-04-30    9
Freq: M, dtype: int32

In [235]:
# truncating preserves frequency
ts.truncate(before = '10/31/2015', after = '12/31/2015')

2015-10-31    3
2015-11-30    4
2015-12-31    5
Freq: M, dtype: int32

In [253]:
# You can truncate in a way that does not preserve frequency
ts[[1, 6, 7]].index

DatetimeIndex(['2015-08-31', '2016-01-31', '2016-02-29'], dtype='datetime64[ns]', freq=None)

In [237]:
# But Pandas will try to preserve frequency automatically whenever possible
ts[0:10:2]

2015-07-31    0
2015-09-30    2
2015-11-30    4
2016-01-31    6
2016-03-31    8
Freq: 2M, dtype: int32